# Lab 3: Solve the Maze using A*

In [ ]:
# Import libraries
from pylab import *
from pandas import *
import numpy
from random import shuffle

In [ ]:
# Generate maze
def make_maze(w, h):
    vis = [[0] * w + [1] for _ in range(h)] + [[1] * (w + 1)]
    ver = [["10"] * w + ['1'] for _ in range(h)] + [[]]
    hor = [["11"] * w + ['1'] for _ in range(h + 1)]
 
    def walk(x, y):
        vis[y][x] = 1
 
        d = [(x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)]
        shuffle(d)
        for (xx, yy) in d:
            if vis[yy][xx]: continue
            if xx == x: hor[max(y, yy)][x] = "10"
            if yy == y: ver[y][max(x, xx)] = "00"
            walk(xx, yy)
 
    walk(numpy.random.randint(w), numpy.random.randint(h))
 
    s = ""
    for (a, b) in zip(hor, ver):
        s += ''.join(a + ['\n'] + b + ['\n'])
        
    M=[]
    for line in s.split("\n"):
        if line!="":
            R=[]
            for e in line:
                R.append(int(e))
            M.append(R)
    return M
 
maze=numpy.array(make_maze(8,8))

In [ ]:
import heapq

# Returns a list of the neighbors of the given square.
# The neighbors are the squares directly above, below, left, and right of the given square.
# Only squares with a value of 0 in the maze are considered to be valid neighbors.
def get_neighbors(maze, square):
    x, y = square
    neighbors = []
    if x > 0 and maze[x - 1][y] == 0:
        neighbors.append((x - 1, y))
    if x < len(maze) - 1 and maze[x + 1][y] == 0:
        neighbors.append((x + 1, y))
    if y > 0 and maze[x][y - 1] == 0:
        neighbors.append((x, y - 1))
    if y < len(maze[0]) - 1 and maze[x][y + 1] == 0:
        neighbors.append((x, y + 1))
    return neighbors

# Backtracks from the end square to find the path from the start square.
# The `parents` dictionary maps each square to its parent square.
# This allows us to follow the chain of parents backwards from the end square to the start square.
def backtrack(parents, end):
    # Create a list to store the path.
    path = []
    # Start at the end square and work backwards.
    current = end
    # Keep looping until we reach the start square.
    while current:
        # Prepend the current square to the path list.
        path.insert(0, current)
        # Set the current square to be its parent.
        current = parents[current]
    # Return the final path.
    return path

# Solves the maze using the A* algorithm.
def a_star(maze, start, end):
    # Create a priority queue to store the squares that we need to visit.
    # The priority queue is used to efficiently find the next square to visit.
    queue = []
    # Push the starting square onto the queue with a priority of 0.
    heapq.heappush(queue, (0, start))

    # Create a dictionary to store the cost of each square.
    # This will allow us to efficiently retrieve the cost of each square.
    costs = {start: 0}

    # Create a dictionary to store the parent of each square.
    # This will allow us to backtrack and find the path once the end square is reached.
    parents = {start: None}

    # Keep looping until the queue is empty or we reach the end square.
    while queue:
        # Get the square with the lowest priority from the queue.
        priority, current = heapq.heappop(queue)

        # If we reached the end square, return the path.
        if current == end:
            return backtrack(parents, end)

        # Get the neighbors of the current square.
        neighbors = get_neighbors(maze, current)

        # Loop over the neighbors.
        for neighbor in neighbors:
            # Calculate the cost of reaching the neighbor.
            # In this case, the cost is always 1 because all moves have the same cost.
            cost = costs[current] + 1

            # If the neighbor has not been visited or if the cost of reaching the neighbor is lower than the current cost, update the cost and parent of the neighbor.
            if neighbor not in costs or cost < costs[neighbor]:
                costs[neighbor] = cost
                priority = cost + abs(end[0] - neighbor[0]) + abs(end[1] - neighbor[1])
                parents[neighbor] = current
                heapq.heappush(queue, (priority, neighbor))

    # If we reach this point, it means that we were unable to find a path from the start square to the end square.
    return None

In [ ]:
# Define start and target locations
startLocation=(1,1)
targetLocation=(15,15)

# Check if the start and target locations are valid
if maze[startLocation[0]][startLocation[1]]==1:
    print('Start location is not valid =', maze[1][1])
else:
    print('Start location is valid =', maze[1][1])
if maze[targetLocation[0]][targetLocation[1]]==1:
    print('Target location is not valid =', maze[15][15])
else:
    print('Target location is valid =', maze[15][15])

# Plot mission
imshow(maze, cmap='binary')
plot(startLocation[1],startLocation[0],'gs')
plot(targetLocation[1],targetLocation[0],'rs')

In [ ]:
# Find the best path with A*
generated_path = a_star(maze, startLocation, targetLocation)

# Plot the path
imshow(maze, cmap='binary')
plot(startLocation[1],startLocation[0],'gs')
plot(targetLocation[1],targetLocation[0],'rs')
plot(transpose(generated_path[1:-1])[1],transpose(generated_path[1:-1])[0],'ko')